In [1]:
!pip install bitarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 14.3 MB/s eta 0:00:00


In [2]:
import random
import time
from bitarray import bitarray
from collections import defaultdict, Counter

In [4]:
ip_table = [
    58, 50, 42, 34, 26, 18, 10, 2,
    60, 52, 44, 36, 28, 20, 12, 4,
    62, 54, 46, 38, 30, 22, 14, 6,
    64, 56, 48, 40, 32, 24, 16, 8,
    57, 49, 41, 33, 25, 17, 9, 1,
    59, 51, 43, 35, 27, 19, 11, 3,
    61, 53, 45, 37, 29, 21, 13, 5,
    63, 55, 47, 39, 31, 23, 15, 7
]

# Inverse Initial Permutation (IP⁻¹) Table
ip_inverse_table = [
    40, 8, 48, 16, 56, 24, 64, 32,
    39, 7, 47, 15, 55, 23, 63, 31,
    38, 6, 46, 14, 54, 22, 62, 30,
    37, 5, 45, 13, 53, 21, 61, 29,
    36, 4, 44, 12, 52, 20, 60, 28,
    35, 3, 43, 11, 51, 19, 59, 27,
    34, 2, 42, 10, 50, 18, 58, 26,
    33, 1, 41, 9, 49, 17, 57, 25
]

# PC1 permutation table (for key scheduling, 64-bit -> 56-bit)
pc1_table = [
    57, 49, 41, 33, 25, 17, 9, 1,
    58, 50, 42, 34, 26, 18, 10, 2,
    59, 51, 43, 35, 27, 19, 11, 3,
    60, 52, 44, 36, 63, 55, 47, 39,
    31, 23, 15, 7, 62, 54, 46, 38,
    30, 22, 14, 6, 61, 53, 45, 37,
    29, 21, 13, 5, 28, 20, 12, 4
]

# Left shift schedule for the key halves
shift_schedule = [
    1, 1, 2, 2,
    2, 2, 2, 2,
    1, 2, 2, 2,
    2, 2, 2, 1
]

# PC2 permutation table (for round-key compression, 56-bit -> 48-bit)
pc2_table = [
    14, 17, 11, 24, 1, 5, 3, 28,
    15, 6, 21, 10, 23, 19, 12, 4,
    26, 8, 16, 7, 27, 20, 13, 2,
    41, 52, 31, 37, 47, 55, 30, 40,
    51, 45, 33, 48, 44, 49, 39, 56,
    34, 53, 46, 42, 50, 36, 29, 32
]

# Expansion (E-box) table (32-bit -> 48-bit)
e_box_table = [
    32, 1, 2, 3, 4, 5,
    4, 5, 6, 7, 8, 9,
    8, 9, 10, 11, 12, 13,
    12, 13, 14, 15, 16, 17,
    16, 17, 18, 19, 20, 21,
    20, 21, 22, 23, 24, 25,
    24, 25, 26, 27, 28, 29,
    28, 29, 30, 31, 32, 1
]

# S-boxes (8 boxes)
s_boxes = [
    # S-box 1
    [
        [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
        [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
        [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
        [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
    ],
    # S-box 2
    [
        [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
        [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
        [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
        [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]
    ],
    # S-box 3
    [
        [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
        [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
        [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
        [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]
    ],
    # S-box 4
    [
        [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
        [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
        [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
        [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]
    ],
    # S-box 5
    [
        [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
        [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
        [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
        [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]
    ],
    # S-box 6
    [
        [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
        [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
        [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
        [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]
    ],
    # S-box 7
    [
        [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
        [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
        [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
        [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]
    ],
    # S-box 8
    [
        [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
        [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
        [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
        [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]
    ]
]

# P-Box table (32-bit -> 32-bit permutation)
p_box_table = [
    16, 7, 20, 21, 29, 12, 28, 17,
    1, 15, 23, 26, 5, 18, 31, 10,
    2, 8, 24, 14, 32, 27, 3, 9,
    19, 13, 30, 6, 22, 11, 4, 25
]

# Some differential analysis often needs the inverse P-box mapping
# (If needed, we define it as well)
ptable_inverse = [
    9, 17, 23, 31, 13, 28, 2, 18,
    24, 16, 30, 6, 26, 20, 10, 1,
    8, 14, 25, 3, 12, 29, 11, 19,
    32, 15, 22, 7, 5, 27, 4, 21
]


In [6]:

def binary_to_hex(binary_str):
    """
    Converts a 64-bit binary string to a 16-hex-digit string.
    """
    if len(binary_str) != 64:
        return "Error: Input string must be exactly 64 bits long."
    integer_value = int(binary_str, 2)
    return format(integer_value, '016x').upper()

def hex_to_binary(hex_str):
    """
    Converts a 16-hex-digit string to a 64-bit binary string.
    """
    if len(hex_str) != 16:
        return "Error: Input string must be exactly 16 hexadecimal digits long."
    integer_value = int(hex_str, 16)
    return format(integer_value, '064b')

def xor_binary_strings(bin_str1, bin_str2):
    """
    Returns XOR of two equal-length binary strings.
    """
    if len(bin_str1) != len(bin_str2):
        raise ValueError("Both strings must be of the same length.")
    return ''.join('1' if c1 != c2 else '0' for c1, c2 in zip(bin_str1, bin_str2))

def str_to_bin(input_string):
    """
    Converts any ASCII string into 64-bit blocks of binary strings.
    Pads with null ('\\x00') if length is not multiple of 8.
    """
    block_size = 8  # 8 characters = 64 bits
    blocks = []
    for i in range(0, len(input_string), block_size):
        block = input_string[i:i + block_size]
        # If the chunk is not of the desired length, pad it with null chars
        if len(block) < block_size:
            block = block.ljust(block_size, '\x00')
        binary_representation = ''.join(format(ord(char), '08b') for char in block)
        blocks.append(binary_representation)
    return blocks

def binary_to_ascii(binary_str):
    """
    Converts a binary string (multiple of 8 bits) into an ASCII string.
    """
    return ''.join([chr(int(binary_str[i:i+8], 2)) for i in range(0, len(binary_str), 8)])

def ip_on_binary_rep(bin_blocks):
    """
    Applies the initial permutation (IP) on each 64-bit block.
    """
    permuted_blocks = []
    for binary_representation in bin_blocks:
        if len(binary_representation) != 64:
            raise ValueError("Each binary representation must be 64 bits long.")
        ip_result = [''] * 64
        for i in range(64):
            ip_result[i] = binary_representation[ip_table[i] - 1]
        permuted_blocks.append(''.join(ip_result))
    return permuted_blocks

In [8]:
def generate_round_keys(key, num_rounds):
    """
    Generates round keys (each 48 bits) for a given ASCII key and desired number of rounds.
    """
    # Convert key to 64-bit binary (first block)
    binary_key_64 = str_to_bin(key)[0]

    # Apply PC1 to reduce from 64 -> 56 bits
    pc1_key_str = ''.join(binary_key_64[bit - 1] for bit in pc1_table)

    # Split into two halves (28 bits each)
    c0 = pc1_key_str[:28]
    d0 = pc1_key_str[28:]

    round_keys = []
    for round_num in range(num_rounds):
        # Circular left shift
        shift_amount = shift_schedule[round_num % len(shift_schedule)]
        c0 = c0[shift_amount:] + c0[:shift_amount]
        d0 = d0[shift_amount:] + d0[:shift_amount]

        # Concatenate and apply PC2 (56 -> 48 bits)
        cd_concatenated = c0 + d0
        round_key = ''.join(cd_concatenated[bit - 1] for bit in pc2_table)
        round_keys.append(round_key)

    return round_keys

In [10]:

def encryption(user_input, key, num_rounds, binary=False):
    """
    Encrypts the user_input using DES-like rounds (num_rounds).

    :param user_input: (str) ASCII plaintext or a single 64-bit block if 'binary=True'
    :param key:        (str) ASCII key (8 chars for 64 bits)
    :param num_rounds: (int) Number of encryption rounds (1..16 typical)
    :param binary:     (bool) If True, user_input is already a 64-bit binary string
    :return: (ciphertext_ascii, last_round_binary)
    """
    if binary:
        # user_input is a single 64-bit block in binary form
        binary_blocks = [user_input]
    else:
        # Convert ASCII to blocks of 64-bit binary
        binary_blocks = str_to_bin(user_input)

    # Generate all round keys
    round_keys = generate_round_keys(key, num_rounds)

    # Optionally apply the initial permutation (if desired)
    # ip_result_blocks = ip_on_binary_rep(binary_blocks)
    # Here, we skip IP for simpler analysis in differential attacks:
    ip_result_blocks = binary_blocks

    final_cipher_blocks = []

    for block in ip_result_blocks:
        lpt = block[:32]
        rpt = block[32:]

        # For each round (Feistel)
        for round_num in range(num_rounds):
            # Expand R
            expanded_r = ''.join(rpt[i - 1] for i in e_box_table)
            # XOR with round key
            # NOTE: Some differential attacks do reverse key usage, but standard DES uses round_keys[round_num]
            round_key_str = round_keys[num_rounds - 1 - round_num]
            xor_result = xor_binary_strings(expanded_r, round_key_str)

            # Split into 8 groups of 6 bits for S-box
            six_bit_groups = [xor_result[i:i+6] for i in range(0, 48, 6)]
            s_box_substitution = ''
            for sbox_i, six_bits in enumerate(six_bit_groups):
                row_bits = int(six_bits[0] + six_bits[-1], 2)
                col_bits = int(six_bits[1:-1], 2)
                val = s_boxes[sbox_i][row_bits][col_bits]
                s_box_substitution += format(val, '04b')

            # Apply P-box
            p_box_out = ''.join(s_box_substitution[i - 1] for i in p_box_table)
            # XOR with L
            new_r = xor_binary_strings(lpt, p_box_out)

            # Feistel swap
            lpt, rpt = rpt, new_r

        # Combine R+L after the final round
        final_block = rpt + lpt  # (No final IP for differential analysis)
        final_cipher_blocks.append(final_block)

    # Convert final blocks from binary to ASCII
    final_cipher_ascii = ''.join(binary_to_ascii(b) for b in final_cipher_blocks)

    # Return the final block's binary form for analysis plus ASCII
    return final_cipher_ascii, final_cipher_blocks[-1]

def decryption(cipher_text, key, num_rounds):
    """
    Standard DES-like decryption with reversed round key order.
    This function uses the standard approach with IP -> rounds -> IP⁻¹
    (But you can adapt for differential analysis by skipping IP if needed.)
    """
    # Convert cipher_text ASCII to 64-bit binary blocks
    cipher_blocks = str_to_bin(cipher_text)

    # Generate all round keys
    round_keys = generate_round_keys(key, num_rounds)
    reversed_keys = list(reversed(round_keys))

    decrypted_blocks = []

    # Apply IP if standard approach:
    # ip_dec_result_blocks = ip_on_binary_rep(cipher_blocks)
    # We skip IP for brevity/differential analysis:
    ip_dec_result_blocks = cipher_blocks

    for block in ip_dec_result_blocks:
        lpt = block[:32]
        rpt = block[32:]

        for round_num in range(num_rounds):
            expanded_r = ''.join(rpt[i - 1] for i in e_box_table)
            round_key_str = reversed_keys[round_num]  # reversed order
            xor_result = xor_binary_strings(expanded_r, round_key_str)

            six_bit_groups = [xor_result[i:i+6] for i in range(0, 48, 6)]
            s_box_substitution = ''
            for sbox_i, six_bits in enumerate(six_bit_groups):
                row_bits = int(six_bits[0] + six_bits[-1], 2)
                col_bits = int(six_bits[1:-1], 2)
                val = s_boxes[sbox_i][row_bits][col_bits]
                s_box_substitution += format(val, '04b')

            p_box_out = ''.join(s_box_substitution[i - 1] for i in p_box_table)
            new_r = xor_binary_strings(lpt, p_box_out)

            lpt, rpt = rpt, new_r

        # Combine R+L
        final_block = rpt + lpt
        decrypted_blocks.append(final_block)

    # Convert from binary to ASCII
    final_plaintext_ascii = ''.join(binary_to_ascii(b) for b in decrypted_blocks)
    # Strip possible null padding
    final_plaintext_ascii = final_plaintext_ascii.rstrip('\x00')
    return final_plaintext_ascii


In [12]:

def generate_binary_string(length=64):
    """
    Generates a random binary string of specified length.
    """
    return ''.join(random.choice('01') for _ in range(length))

def generate_pairs(desired_xor, num_pairs):
    """
    Generates num_pairs pairs of (bin1, bin2) each of length 64 bits,
    such that bin1 XOR bin2 == desired_xor.
    """
    pairs = []
    for _ in range(num_pairs):
        bin1 = generate_binary_string()
        bin2 = xor_binary_strings(bin1, desired_xor)
        pairs.append((bin1, bin2))
    return pairs

def check_right_pairs(pairs_list, desired_xor, key, round_num):
    """
    Given a list of (bin1, bin2) pairs, encrypt both with 'key' for 'round_num' rounds,
    then count how many produce the 'desired_xor' in the output.
    """
    right_pairs = []
    for bin1, bin2 in pairs_list:
        _, out1 = encryption(bin1, key, round_num, binary=True)
        _, out2 = encryption(bin2, key, round_num, binary=True)
        out_xor = xor_binary_strings(out1, out2)
        if out_xor == desired_xor:
            right_pairs.append((bin1, bin2))

    ratio_right_pairs = len(right_pairs) / len(pairs_list) if pairs_list else 0
    return right_pairs, ratio_right_pairs

def sbox_entry(six_bit_str, sbox_index):
    """
    Applies the (sbox_index)th s-box to a 6-bit input string.
    sbox_index is 0-based in this function.
    """
    row_bits = int(six_bit_str[0] + six_bit_str[-1], 2)
    col_bits = int(six_bit_str[1:-1], 2)
    s_value = s_boxes[sbox_index][row_bits][col_bits]
    return format(s_value, '04b')

def four_round_attack(pair, sbox_list, characteristic_xor):
    """
    Simple demonstration of a 4-round differential attack approach.
    We skip the real IP and final IP⁻¹ for clarity.

    :param pair: (bin1, bin2) each 64 bits
    :param sbox_list: which S-box indices (1-based) we suspect for the characteristic
    :param characteristic_xor: 64-bit string representing input difference
    :return: a dictionary { sbox_number : { candidate_6bit_key : count } }
    """
    # Initialize dictionary only for specified S-boxes
    key_dict = {sbox: {} for sbox in sbox_list}

    bin1, bin2 = pair
    # Encrypt with a "dummy" key of all zeros (or skip if you prefer)
    # for differential analysis
    _, enc1_bin = encryption(bin1, "00000000", 4, binary=True)
    _, enc2_bin = encryption(bin2, "00000000", 4, binary=True)

    # Output XOR after 4 rounds
    out_xor = xor_binary_strings(enc1_bin, enc2_bin)

    # Right half expansions
    right_enc1 = enc1_bin[32:]
    right_enc2 = enc2_bin[32:]
    expanded_enc1 = ''.join(right_enc1[i - 1] for i in e_box_table)
    expanded_enc2 = ''.join(right_enc2[i - 1] for i in e_box_table)

    # Split expansions into 8 groups of 6 bits
    grouped_enc1 = [expanded_enc1[i:i+6] for i in range(0, 48, 6)]
    grouped_enc2 = [expanded_enc2[i:i+6] for i in range(0, 48, 6)]

    # The output XOR's left 32 bits typically feed into F function's next round
    # but here we do a simplified approach. We can also combine P-box if needed.
    l_prime = out_xor[:32]
    # Optionally invert P if characteristic is measured pre-P-box
    l_prime_invP = ''.join(l_prime[i - 1] for i in ptable_inverse)
    grouped_l_prime = [l_prime_invP[i:i+4] for i in range(0, 32, 4)]

    # Check possible 6-bit subkeys for each S-box
    for possible_key_int in range(64):
        possible_key_bin = format(possible_key_int, '06b')
        for sbox in sbox_list:
            idx = sbox - 1  # zero-based

            # XOR with candidate key
            SI1 = xor_binary_strings(grouped_enc1[idx], possible_key_bin)
            SI2 = xor_binary_strings(grouped_enc2[idx], possible_key_bin)

            # S-box outputs
            SO1 = sbox_entry(SI1, idx)
            SO2 = sbox_entry(SI2, idx)

            # If the XOR of S-box outputs matches our expected difference (after P)
            if xor_binary_strings(SO1, SO2) == grouped_l_prime[idx]:
                if possible_key_bin in key_dict[sbox]:
                    key_dict[sbox][possible_key_bin] += 1
                else:
                    key_dict[sbox][possible_key_bin] = 1

    return key_dict

def six_round_attack(pair, sbox_list, characteristic_xor, real_key_for_encryption="00000000"):
    """
    Demonstration of a 6-round differential attack approach.
    We assume the same idea: skip real IP, final IP⁻¹, etc.

    :param pair: (bin1, bin2) each 64 bits
    :param sbox_list: which S-box indices (1-based) we suspect for the characteristic
    :param characteristic_xor: 64-bit string representing input difference
    :param real_key_for_encryption: actual key used for encryption (ASCII, 8 chars)
    :return: a dictionary { sbox_number : { candidate_6bit_key : count } }
    """
    key_dict = {sbox: {} for sbox in sbox_list}
    bin1, bin2 = pair

    # Encrypt both with the real_key_for_encryption for 6 rounds
    _, enc1_bin = encryption(bin1, real_key_for_encryption, 6, binary=True)
    _, enc2_bin = encryption(bin2, real_key_for_encryption, 6, binary=True)

    # Overall output XOR
    out_xor = xor_binary_strings(enc1_bin, enc2_bin)

    # For 6-round differential characteristic, we might incorporate P-box differences, etc.
    # Let's define a simplified approach:
    left_out = out_xor[:32]
    right_out = out_xor[32:]

    # This characteristic might say left_out XOR c_prime = next F difference, etc.
    # We'll do a simplified approach:
    c_prime = characteristic_xor[32:]
    F_prime = xor_binary_strings(left_out, c_prime)
    # Invert P for the left side difference
    F_prime_invP = ''.join(F_prime[i - 1] for i in ptable_inverse)
    grouped_fprime = [F_prime_invP[i:i+4] for i in range(0, 32, 4)]

    # Expand the final right half to see S-box inputs
    right_enc1 = enc1_bin[32:]
    right_enc2 = enc2_bin[32:]
    expanded_enc1 = ''.join(right_enc1[i - 1] for i in e_box_table)
    expanded_enc2 = ''.join(right_enc2[i - 1] for i in e_box_table)
    grouped_enc1 = [expanded_enc1[i:i+6] for i in range(0, 48, 6)]
    grouped_enc2 = [expanded_enc2[i:i+6] for i in range(0, 48, 6)]

    # Try all candidate 6-bit subkeys
    for candidate_int in range(64):
        candidate_key_bin = format(candidate_int, '06b')
        for sbox in sbox_list:
            idx = sbox - 1
            SI1 = xor_binary_strings(grouped_enc1[idx], candidate_key_bin)
            SI2 = xor_binary_strings(grouped_enc2[idx], candidate_key_bin)
            SO1 = sbox_entry(SI1, idx)
            SO2 = sbox_entry(SI2, idx)

            if xor_binary_strings(SO1, SO2) == grouped_fprime[idx]:
                if candidate_key_bin in key_dict[sbox]:
                    key_dict[sbox][candidate_key_bin] += 1
                else:
                    key_dict[sbox][candidate_key_bin] = 1

    return key_dict

def aggregate_results(all_results):
    """
    Aggregates multiple dictionaries of the form:
    [{ sbox_number : { key_bin : count } }, ...]
    into a single dictionary with summed counts.
    """
    aggregated_dict = {}
    for result in all_results:
        for sbox, keys in result.items():
            if sbox not in aggregated_dict:
                aggregated_dict[sbox] = {}
            for k, count in keys.items():
                if k in aggregated_dict[sbox]:
                    aggregated_dict[sbox][k] += count
                else:
                    aggregated_dict[sbox][k] = count
    return aggregated_dict

def find_most_frequent_keys(aggregated_dict):
    """
    For each sbox_number in aggregated_dict,
    find the (key_bin, count) with the maximum count.
    Returns a dict { sbox_number : (key_bin, max_count) }
    """
    most_frequent = {}
    for sbox, keys in aggregated_dict.items():
        # pick the key with maximum count
        best_key = max(keys, key=keys.get)
        most_frequent[sbox] = (best_key, keys[best_key])
    return most_frequent

In [18]:
print("=== 4-ROUND DIFFERENTIAL ATTACK DEMO ===")

# 1) Define a 4-round characteristic in hex (for example):
char_xor_4round_hex = "2000000000000000"
char_xor_4round_bin = hex_to_binary(char_xor_4round_hex)  # 64 bits

# 2) Generate pairs that satisfy P⊕P* = char_xor_4round_bin
pairs_4 = generate_pairs(char_xor_4round_bin, 1024)  # small number for demonstration

# 3) Check how many are actually right pairs under some test key
test_key_4 = "abcdefgh"  # 8-char ASCII
right_pairs_4, ratio_4 = check_right_pairs(pairs_4, char_xor_4round_bin, test_key_4, 1)
print(f"Number of right pairs for 1-round test key = {len(right_pairs_4)}, ratio = {ratio_4}")

# 4) Perform the simplified 4-round attack on each "right pair" using a dummy encryption (0 key).
sbox_list_4 = [2, 3, 4, 5, 6, 7, 8]  # Example subset of S-boxes
all_results_4 = []

for rp in right_pairs_4:
    attack_res = four_round_attack(rp, sbox_list_4, char_xor_4round_bin)
    all_results_4.append(attack_res)

# 5) Aggregate results
agg_4 = aggregate_results(all_results_4)
mfreq_4 = find_most_frequent_keys(agg_4)

print("Most frequent candidate subkeys for each S-box (4-round):")
for sbox_num, (key_bin, count) in mfreq_4.items():
    print(f" S-box {sbox_num}: key candidate = {key_bin}, count = {count}")

print("\n=== 4-ROUND DEMO COMPLETE ===\n")

# ============================
#      6-ROUND EXAMPLE
# ============================
print("=== 6-ROUND DIFFERENTIAL ATTACK DEMO ===")

# 1) Two example 6-round characteristics in hex
char_xor_6round_hex_1 = "4008000004000000"
char_xor_6round_hex_2 = "0020000800000400"

char_xor_6round_bin_1 = hex_to_binary(char_xor_6round_hex_1)
char_xor_6round_bin_2 = hex_to_binary(char_xor_6round_hex_2)

# 2) Generate pairs
pairs_6_1 = generate_pairs(char_xor_6round_bin_1, 2**20)  # small for demo
pairs_6_2 = generate_pairs(char_xor_6round_bin_2, 2**20)  # small for demo

# 3) Check right pairs under some test key for 3 rounds
test_key_6 = "hellowww"

rps_6_1, ratio_6_1 = check_right_pairs(pairs_6_1, char_xor_6round_bin_1, test_key_6, 3)
rps_6_2, ratio_6_2 = check_right_pairs(pairs_6_2, char_xor_6round_bin_2, test_key_6, 3)

print(f"Right pairs (1st char): {len(rps_6_1)}, ratio: {ratio_6_1}")
print(f"Right pairs (2nd char): {len(rps_6_2)}, ratio: {ratio_6_2}")

# 4) Attack each right pair with a simplified 6-round approach
sbox_list_6_1 = [2, 5, 6, 7, 8]   # Example subset of S-boxes for characteristic 1
sbox_list_6_2 = [1, 2, 4, 5, 6]   # Example subset of S-boxes for characteristic 2

all_results_6_1 = []
for rp in rps_6_1:
    attack_res_6_1 = six_round_attack(rp, sbox_list_6_1, char_xor_6round_bin_1, real_key_for_encryption=test_key_6)
    all_results_6_1.append(attack_res_6_1)

all_results_6_2 = []
for rp in rps_6_2:
    attack_res_6_2 = six_round_attack(rp, sbox_list_6_2, char_xor_6round_bin_2, real_key_for_encryption=test_key_6)
    all_results_6_2.append(attack_res_6_2)

# 5) Aggregate results
agg_6_1 = aggregate_results(all_results_6_1)
agg_6_2 = aggregate_results(all_results_6_2)


def safe_find_most_frequent_keys(aggregated_dict):
    """
    Similar to find_most_frequent_keys, but safely handles cases
    where a dictionary or an S-box entry might be empty.
    Returns { sbox : (key, count) } only if data exists for that sbox.
    """
    most_frequent = {}
    for sbox, keys in aggregated_dict.items():
        if not keys:  # If the dictionary for this sbox is empty, skip
            continue
        # Otherwise pick the key with the maximum count
        best_key = max(keys, key=keys.get)
        most_frequent[sbox] = (best_key, keys[best_key])
    return most_frequent


mfreq_6_1 = safe_find_most_frequent_keys(agg_6_1)
mfreq_6_2 = safe_find_most_frequent_keys(agg_6_2)

print("Most frequent candidate subkeys for S-boxes in 1st 6-round characteristic:")
if not mfreq_6_1:
    print("  [No data found or no non-empty S-box counters.]")
else:
    for sbox_num, (key_bin, count) in mfreq_6_1.items():
        print(f"  S-box {sbox_num}: candidate = {key_bin}, occurrences = {count}")

print("\nMost frequent candidate subkeys for S-boxes in 2nd 6-round characteristic:")
if not mfreq_6_2:
    print("  [No data found or no non-empty S-box counters.]")
else:
    for sbox_num, (key_bin, count) in mfreq_6_2.items():
        print(f"  S-box {sbox_num}: candidate = {key_bin}, occurrences = {count}")

print("\n=== 6-ROUND DEMO COMPLETE ===")


=== 4-ROUND DIFFERENTIAL ATTACK DEMO ===
Number of right pairs for 1-round test key = 1024, ratio = 1.0
Most frequent candidate subkeys for each S-box (4-round):
 S-box 2: key candidate = 000010, count = 1024
 S-box 3: key candidate = 110010, count = 1024
 S-box 4: key candidate = 101100, count = 1024
 S-box 5: key candidate = 010001, count = 1024
 S-box 6: key candidate = 000000, count = 528
 S-box 7: key candidate = 000001, count = 510
 S-box 8: key candidate = 000000, count = 544

=== 4-ROUND DEMO COMPLETE ===

=== 6-ROUND DIFFERENTIAL ATTACK DEMO ===
Right pairs (1st char): 65694, ratio: 0.06265068054199219
Right pairs (2nd char): 65202, ratio: 0.06218147277832031
Most frequent candidate subkeys for S-boxes in 1st 6-round characteristic:
  S-box 2: candidate = 000100, occurrences = 4226
  S-box 5: candidate = 110111, occurrences = 4251
  S-box 6: candidate = 100011, occurrences = 4219
  S-box 7: candidate = 111101, occurrences = 4216
  S-box 8: candidate = 000011, occurrences = 422